## Imports

In [ ]:
# import lmdb
# import torch
from DataClasses import lmdb_dataset
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
import pickle
from tqdm import tqdm
import os

## Open datasets

In [ ]:
root_dir = "../../ocp_datasets/data/is2re/"
file = 'data_mod.lmdbz'
datasets = {
    
    'train_10k':     '10k/train/',
    'train_100k':    '100k/train/',
    'train_all':     'all/train/',

    'val_id':        'all/val_id/',
    'val_ood_ads':   'all/val_ood_ads/',
    'val_ood_cat':   'all/val_ood_cat/',
    'val_ood_both':  'all/val_ood_both/',

    'test_id':       'all/test_id/',
    'test_ood_ads':  'all/test_ood_ads/',
    'test_ood_cat':  'all/test_ood_cat/',
    'test_ood_both': 'all/test_ood_both/'
}

datasets = {k:(root_dir + v + file) for k, v in datasets.items()}

In [ ]:
train_10k = lmdb_dataset(datasets['train_10k'])

In [ ]:
train_all = lmdb_dataset(datasets['train_all'])

In [ ]:
test_challenge = lmdb_dataset('/share/catalyst/ocp_datasets/is2re_test_challenge_2021/data_mod1_0_50039.lmdbz')

In [ ]:
test_challenge.describe(3228)

In [ ]:
type(test_challenge[3219])

#### Calculate neighbours for each edge

In [ ]:
def t2str(tensor, index):
    return tensor.T[index].numpy()

system = train_10k[0]
edge_angles = system['edge_angles'][0]
edge_index = system['edge_index_new']

edge_angles['nb'] = list(map(tuple,t2str(edge_index, edge_angles.index)))
edge_angles

## Analyzing dataset

#### Describe dataset function (already used as method .describe() in lmdb_dataset)

#### create DataFrame of .natom keys

In [ ]:
# create DataFrame of .natom keys
def natom_hist(dataset):
    from collections import defaultdict
    import pandas as pd
    sec = time.time()
    dic = defaultdict(int)
    for struct in dataset:
        dic[struct.natoms] += 1
    print(f'done for: {time.time() - sec:.2f} s')

    dic_mod = {k:[v] for k,v in dict(dic).items()}
    return pd.DataFrame.from_dict(dic_mod, orient='index', columns=['N']).sort_index()


# TODO multiprocessing edition of function
# from multiprocessing import Pool
# def natom_hist(dataset):
#     from collections import defaultdict
#     import pandas as pd
#     sec = time.time()
#     dic = defaultdict(int)
#     for struct in dataset:
#         dic[struct.natoms] += 1
#     print(time.time() - sec)

#     dic_mod = {k:[v] for k,v in dict(dic).items()}
#     return pd.DataFrame.from_dict(dic_mod, orient='index', columns=['N']).sort_index()

In [ ]:
# long time to run (~20 min)
train_10k_df = natom_hist(train_10k)

#### Sparce distribution

In [ ]:
def df_distr(dic):
    dic = dic.to_dict()
    key = list(dic.keys())[0]
    dic1 = {key:{i:0 for i in range(1, 301)}}
    
    for k, v in dic[key].items():
        dic1[key][k] = v
    return pd.DataFrame.from_dict(dic1)

In [ ]:
frames = (train_10k_df,)
frames_distr = [0]*len(frames)

In [ ]:
for i, frame in enumerate(frames):
    frames_distr[i] = df_distr(frame)

In [ ]:
df_all = pd.concat(frames_distr, axis=1)
df_all.columns = ['df_train_all', 'dataset_test_id_df', 'dataset_train_10k_df', 'dataset_val_ood_both_df']
df_all_norm = df_all.fillna(0)
df_all_norm = df_all_norm.apply(lambda x: x / x.max())
df_all_norm.plot()

In [ ]:
dataset_train_keys_10k_keys = set(dataset_train_10k[0].keys)
dataset_val_ood_both_keys = set(dataset_val_ood_both[0].keys)
dataset_test_keys_id_keys = set(dataset_test_id[0].keys)

In [ ]:
print(dataset_train_keys_10k_keys, dataset_val_ood_both_keys, dataset_test_keys_id_keys, sep='\n')
print(dataset_train_keys_10k_keys - dataset_test_keys_id_keys)

In [ ]:
print(dataset_train_10k[0])
# for i in dataset_train_keys_10k_keys:
#         print((i, getattr(dataset_train_10k[0], i)))

for i in dataset_train_keys_10k_keys:
        temp = getattr(dataset_train_10k[0], i)
        print(i, temp)
#         if type(temp) not in [float, int]:
#             print(temp.shape)
#         else: print(temp)

In [ ]:
dataset_train_10k[0]['y_relaxed']

In [ ]:
from collections import Counter
# Counter(list(dataset_train_10k[0].atomic_numbers))
dataset_train_10k[0].atomic_numbers[0].item()
Counter([dataset_train_10k[0].atomic_numbers[i].item() for i in range(dataset_train_10k[0].atomic_numbers.shape[0])])
# compare = [("\n".join((getattr(dataset_train_10k[0], i), getattr(dataset_test[0], i)))) for i in dataset_test[0].keys]

In [ ]:
dataset_train_10k[0].edge_index.shape

In [ ]:
dataset_train_10k[0].natoms

In [ ]:
86**2/2

In [ ]:
print(dataset_train_10k[0])

In [1]:
import pandas as pd

dic = pd.read_pickle("../../ocp_datasets/oc20_data_mapping.pkl")
# or use online https://dl.fbaipublicfiles.com/opencatalystproject/data/oc20_data_mapping.pkl

df = pd.DataFrame.from_dict(dic, orient='index')

In [2]:
df.head(10)

,bulk_id,ads_id,bulk_mpid,bulk_symbols,ads_symbols,miller_index,shift,top,adsorption_site
random2181546,6510,69,mp-22179,Si2Ti2Y2,*N2,"(2, 0, 1)",0.145,True,"((4.5, 12.85, 16.13),)"
random378384,2742,49,mp-9254,Al2Te5,*CHOCH2OH,"(0, 2, 1)",0.031,False,"((1.84, 6.17, 19.13),)"
random1283307,3219,30,mp-1226096,Co7Ge4,*CHO*CHO,"(0, 0, 1)",0.185,True,"((0.25, 2.07, 25.26), (1.25, 1.11, 24.84))"
random2061443,3381,79,mp-972941,Na2Bi6,*OHNH2,"(2, 1, 2)",0.050,False,"((6.22, 2.94, 26.1),)"
random2070408,2889,1,mp-1601,RuTa,*H,"(1, 1, 0)",0.250,True,"((7.58, 5.29, 19.72),)"
random2069970,7892,81,mp-567196,Cs3Sb2Cl9,*CN,"(1, 0, 1)",0.165,False,"((4.22, 1.86, 27.86),)"
random2426372,1431,60,mp-561320,Pb2S2,*OHNNCH3,"(2, 2, 1)",0.241,True,"((4.09, 21.6, 28.5),)"
random2049408,3760,56,mp-30933,Zr6Bi6,*CHOHCH2OH,"(0, 1, 0)",0.250,True,"((1.95, 2.21, 22.46),)"
random1170099,2302,73,mp-249,Al2Hf2,*NONH,"(2, 2, 1)",0.133,False,"((6.37, 4.22, 18.83),)"
random1530103,3589,73,mp-621624,Tl21Sb6,*NONH,"(2, 1, 0)",0.059,True,"((6.1, 2.21, 35.52),)"


In [3]:
df.loc['random2472718']

bulk_id                                425
ads_id                                  74
bulk_mpid                          mp-1791
bulk_symbols                       Al20Rh8
ads_symbols                            *NH
miller_index                     (2, 1, 1)
shift                                0.118
top                                   True
adsorption_site    ((10.07, 0.67, 18.05),)
Name: random2472718, dtype: object

### Test challenge dataset

In [ ]:
# адрес датасета
test_dataset_path = "../../ocp_datasets/data/is2re_test_challenge_2021/merge/data_mod1.lmdbz"
test_dataset_path_mod1 = "../../ocp_datasets/data/is2re_test_challenge_2021/data.lmdb"

In [ ]:
def dataset(path):
    real_path = os.path.expanduser(path)
    dataset = lmdb_dataset(train_dataset_file_path)
    print(len(dataset))
    return dataset

In [ ]:
test_dataset = dataset(test_dataset_path)
test_dataset_mod1 = dataset(test_dataset_path_mod1)

#### Compare sid of original and converted datasets

In [ ]:
def set_sid(dataset):
    list_done = []
    for ii, sys in enumerate(dataset):
        list_done.append(sys['sid'])

        if ii%10000==0:
            print(ii)
    return set(list_done)

In [ ]:
sid = set_sid(test_dataset)
sid_mod1 = set_sid(test_dataset_mod1)

In [ ]:
print(sid ^ sid_mod1)

#### Find all items with all y_relaxed

In [ ]:
def y_relaxed_list(dataset):
    n = []
    for ii, sys in enumerate(dataset):
        if 'y_relaxed' in sys.keys:
            n.append(ii)

        if ii%10000==0:
            print(ii)
    print('done', len(n))
    return n

In [ ]:
y_relaxed = y_relaxed_list(test_dataset)
y_relaxed_mod1 = y_relaxed_list(test_dataset_mod1)

## Structure of element to VASP

In [ ]:
test_dataset_path = "../../ocp_datasets/data/is2re_test_challenge_2021/data_mod1.lmdbz"
train_dataset_file_path = os.path.expanduser(test_dataset_path)
train_dataset_file = lmdb_dataset(train_dataset_file_path)

In [ ]:
import numpy as np
from mendeleev import element

def print_var_name(variable):
     for name in locals():
        if eval(name) == variable:
            print(name)

def getAtomSequence (sequence):
    result = list([[sequence[0], 1]])
    for i in range(1, len(sequence)):
        if sequence[i] == result[-1][0]:
            result[-1][1] += 1
        else:
            result.append([sequence[i], 1])
    return dict(result)

def structureToVASP(structure, file='POSCAR', str_name='structure', relaxed=False):
    with open(f'{str_name}_POSCAR{"_relaxed" if relaxed else ""}', 'w') as f:
        f.write(str_name + '\n')
        f.write(str(1.0) + '\n')
        for axis in np.array(structure['cell'][0]):
            for i in range(3):
                f.write(str(axis[i]) + '   ')
                if i == 2:
                    f.write('\n')
        atoms = getAtomSequence(np.array(structure['atomic_numbers'], dtype=int))
        for k in atoms.keys():
            f.write('   ' + element(round(k)).symbol)
        f.write('\n')
        for v in atoms.values():
            f.write('   ' + str(round(v)))
        f.write('\n')
        f.write('Cartesian\n')
        for position in np.array(structure['pos'] if not relaxed else structure['pos_relaxed']):
            for i in range(3):
                f.write(str(position[i]) + '   ')
                if i == 2:
                    f.write('\n')          
    return None

i = 64516
PATH = 'POSCAR/'
for relaxed in [True, False] :
    structureToVASP(train_dataset_file[i], relaxed=relaxed, str_name=f'{PATH}train_all[{i}]')

## PyG training

In [ ]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')

In [ ]:
import time
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

sec = time.time()

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

model = Net()
data = dataset[0]
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    if epoch % int(200/10) == 0:
        print(epoch,':', time.time() - sec)

print('Total:', time.time() - sec)

model.eval()
_, pred = model(data).max(dim=1)
correct = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / int(data.test_mask.sum())
print('Accuracy: {:.4f}'.format(acc))